<a href="https://colab.research.google.com/github/Shashwat17-vit/STAT453_GANs/blob/main/FineTune_GPT3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
df = pd.read_csv("shashwat_qna_expanded.csv")
df.head()
df2 = pd.read_csv("shashwat_qna_test_expanded.csv")
df2.head()

,question,answer,category
0,What is Shashwat Negi's full name?,Shashwat Negi.,General
1,Where is Shashwat Negi currently based?,"Shashwat Negi is currently based in Madison, W...",General
2,What university is Shashwat currently attending?,He is attending the University of Wisconsin-Ma...,Education
3,What is Shashwat Negi's current degree program?,He is pursuing a Master of Science in Data Sci...,Education
4,When is Shashwat expected to graduate?,He is expected to graduate in May 2026.,General


In [25]:
def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        json_response = '{"Top Category": "' + row['category'] + '", "Sub Category": "' + row['answer'] + '"}'
        fine_tuning_data.append({
            "messages": [
                {"role": "user", "content": row['question']},
                {"role": "system", "content": json_response}
            ]
        })
    return fine_tuning_data

In [26]:
from sklearn.model_selection import train_test_split
dataset = pd.read_csv("shashwat_qna_expanded.csv")
converted_data=convert_to_gpt35_format(dataset)
converted_data[0]['messages']

[{'role': 'user', 'content': "What is Shashwat Negi's full name?"},
 {'role': 'system',
  'content': '{"Top Category": "General", "Sub Category": "Shashwat Negi."}'}]

In [23]:
import pandas as pd

# Convert list of dictionaries to DataFrame
converted_data = pd.DataFrame(converted_data)

# Now check the class distribution
print(converted_data['Top Category'].value_counts())

KeyError: 'Top Category'

In [18]:


from sklearn.model_selection import train_test_split
dataset = pd.read_csv("shashwat_qna_expanded.csv")
converted_data=convert_to_gpt35_format(dataset)

# Stratified splitting. Assuming 'Top Category' can be used for stratification
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    stratify=dataset['category'],
    random_state=42  # for reproducibility
)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')

training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="your_open_ai_key")

# Upload Training and Validation Files
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

# Create Fine-Tuning Job
suffix_name = "yt_tutorial"
response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def format_test(row):
    formatted_message = [{"role": "user", "content": row['Support Query']}]
    return formatted_message

def predict(test_messages, fine_tuned_model_id):
    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
    )
    return response.choices[0].message.content

def store_predictions(test_df, fine_tuned_model_id):
    test_df['Prediction'] = None
    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        test_df.at[index, 'Prediction'] = prediction_result

    test_df.to_csv("predictions.csv")